In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [3]:
#convert merged data to dataframe.
comb_data = pd.DataFrame(school_data_complete)
# comb_data.head()

In [4]:
#Unique count of Schools
schools_count = comb_data["school_name"].nunique()
schools_count

15

In [5]:
#count of students
students_count = comb_data["Student ID"].nunique()
students_count

39170

In [6]:
#Total Budget
budget_school_ID = comb_data.groupby(["School ID"]).budget.mean().reset_index()
budget_sum = budget_school_ID["budget"].sum()
budget_sum

24649428

In [7]:
#Average Math Score
avg_math_score = round(comb_data["math_score"].mean(),2)
avg_math_score

78.99

In [8]:
#Average Reading Score
avg_read_score = round(comb_data["reading_score"].mean(),2)
avg_read_score

81.88

In [9]:
#percentage of students passing Math
stu_math_pass = comb_data["math_score"]
stu_pass_math = (stu_math_pass[comb_data.math_score > 69].count())/students_count
stu_pass_math

0.749808526933878

In [10]:
#percentage of students passing Reading
stu_math_read = comb_data["reading_score"]
stu_math_read = (stu_math_read[comb_data.reading_score > 69].count())/students_count
stu_math_read

0.8580546336482001

In [11]:
# All students passing both Math and Reading
pass_read_math = (comb_data[(comb_data["math_score"] > 69) & (comb_data["reading_score"] > 69)].count().unique())
stus_passing = pass_read_math/students_count
stus_passing

array([0.65172326])

In [12]:
# put together the district summary
District_Summary_df = pd.DataFrame({
    "Total Schools" : schools_count,
    "Total Students" : students_count,
    "Total Budget" : budget_sum,
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_read_score,
    "% Passing Math" : stu_pass_math,
    "% Passing Reading" : stu_math_read,
    "% Overall Passing" : stus_passing
})
District_Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.99,81.88,0.749809,0.858055,0.651723


In [13]:
# reassigning the summary for formatting purposes 
District_Summary = District_Summary_df[[
    "Total Schools",
    "Total Students",
    "Total Budget" ,
    "Average Math Score" ,
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing" 
]]

# format columns
District_Summary["Total Students"] = District_Summary["Total Students"].map("{:,}".format)
District_Summary["Total Budget"] = District_Summary["Total Budget"].map("${:,.2f}".format)
District_Summary["% Passing Math"] = District_Summary["% Passing Math"].map("{:,.2%}".format)
District_Summary["% Passing Reading"] = District_Summary["% Passing Reading"].map("{:,.2%}".format)
District_Summary["% Overall Passing"] = District_Summary["% Overall Passing"].map("{:,.2%}".format)

District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


In [14]:
# Starting School summary
# isolating School name and type
# set index to type & Name - based on the school data only
sch_type = school_data.set_index(["school_name"])["type"]

In [15]:
#total students by school
stu_by_sch_cnt = comb_data["school_name"].value_counts()
# stu_by_sch_cnt.head()

In [16]:
# school budget - Groupby is used in a sense of aggregating around that column
budget_by_sch = comb_data.groupby(["school_name"]).mean()["budget"]
# budget_by_sch.head()

In [17]:
# Per student budget - very simple. we know the budget for each school and how many students attend each school
budget_by_stu = budget_by_sch/stu_by_sch_cnt
# budget_by_stu.head()

In [18]:
# average math score by school - same formula as student budget - but we need to find mean of math scores
avg_math_school = comb_data.groupby(["school_name"]).mean()["math_score"]
# avg_math_school.head()

In [19]:
# average reading score by school - same formula as student budget - but we need to find mean of math scores
avg_read_school = comb_data.groupby(["school_name"]).mean()["reading_score"]
# avg_read_school.head()

In [20]:
# % passing math
stu_pass_math = comb_data[comb_data["math_score"] > 69]  #dataframe of students passing math
numPassing_math = stu_pass_math.groupby(["school_name"]).count()["Student ID"] # count the number of students within new df
# numPassing_math

# divide number of students passing math by total number of students per school
per_pass_math = numPassing_math/stu_by_sch_cnt
# per_pass_math.head()

In [21]:
# % passing reading
# follow same logic as we did above
stu_pass_read = comb_data[comb_data["reading_score"] > 69]  #dataframe of students passing reading
numPassing_read = stu_pass_read.groupby(["school_name"]).count()["Student ID"] # count the number of students within new df

# divide number of students passing reading by total number of students per school
per_pass_read = numPassing_read/stu_by_sch_cnt
# per_pass_read.head()

In [22]:
stu_passing = comb_data[(comb_data["reading_score"] > 69)&(comb_data["math_score"] > 69)] #dataframe of students passing reading and math
num_stu_passing = stu_passing.groupby(["school_name"]).count()["Student ID"]    # count the # of students within new df

per_overall_pass = num_stu_passing/stu_by_sch_cnt
# per_overall_pass.head()

In [23]:
#put everthing together in a summary budget_by_sch
school_summary_df = pd.DataFrame({
    "School Type" : sch_type,
    "Total Students" : stu_by_sch_cnt,
    "Total School Budget" : budget_by_sch,
    "Per Student Budget" : budget_by_stu,
    "Average Math Score" : avg_math_school,
    "Average Reading Score" : avg_read_school,
    "% Passing Math" : per_pass_math,
    "% Passing Reading" : per_pass_read,
    "% Overall Passing" : per_overall_pass
})


In [24]:
# reassigning the summary for formatting purposes 
school_summary = school_summary_df[[
   "School Type",
    "Total Students",
    "Total School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing" 
]]

# format columns
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:,.2%}".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:,.2%}".format)
school_summary["% Overall Passing"] = school_summary["% Overall Passing"].map("{:,.2%}".format)

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


In [25]:
# Sort and display the top five performing schools by % overall passing.
df_sort_top = school_summary.sort_values(by=["% Overall Passing"],ascending=False)
df_sort_top.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


In [26]:
# Sort and display the five worst-performing schools by % overall passing.
df_sort_bot = school_summary.sort_values(by=["% Overall Passing"])
df_sort_bot.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


In [27]:
# calling the original data merge - for visualization for next analysis
# comb_data.head()

In [28]:
# Math Scores by Grade
# This can be done easily with a simple pivot of the data
math_score_pivot = pd.pivot_table(comb_data, values="math_score", index="school_name",columns="grade")
math_score_pivot = math_score_pivot[['9th','10th','11th','12th']] # organize the column names
math_score_pivot

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [29]:
# Reading Scores by Grade
# This can be done easily with a simple pivot of the data
read_score_pivot = pd.pivot_table(comb_data, values="reading_score", index="school_name",columns="grade")
read_score_pivot = read_score_pivot[['9th','10th','11th','12th']] 
read_score_pivot

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [35]:
school_summary_df.head(2)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.970398,0.913348


In [49]:
# Scores by School Spending
bins_budget = [0,584.99 ,629.99, 644.99, 679.99] # create bins to cut the data
group_budget = ["<$585","$585-630","$630-645","$645-680"] # what values will be contained in the bins

# subset of school summary dataframe
score_budget = school_summary_df[["Per Student Budget",
                                "Average Math Score",
                               "Average Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing"                              
                                                             ]]
# Creating a new column and adding the bins in based on the cut values
score_budget["Spending Ranges (Per Student)"] = pd.cut(score_budget["Per Student Budget"],bins_budget,labels=group_budget,include_lowest=True)

# grouping the new data set based on the newly created column
score_budget_df = round(score_budget.groupby("Spending Ranges (Per Student)").mean(),3)
score_budget_df[["Average Math Score",
                               "Average Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing"]]

# formatting
score_budget_df["% Passing Math"] = score_budget_df["% Passing Math"].map("{:,.2%}".format)
score_budget_df["% Passing Reading"] = score_budget_df["% Passing Reading"].map("{:,.2%}".format)
score_budget_df["% Overall Passing"] = score_budget_df["% Overall Passing"].map("{:,.2%}".format)


score_budget_df

<ipython-input-49-2f86e43d743e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_budget["Spending Ranges (Per Student)"] = pd.cut(score_budget["Per Student Budget"],bins_budget,labels=group_budget,include_lowest=True)


,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,,
<$585,581.000,83.455,83.934,93.50%,96.60%,90.40%
$585-630,615.500,81.900,83.155,87.10%,92.70%,81.40%
$630-645,639.500,78.519,81.624,73.50%,84.40%,62.90%
$645-680,652.333,76.997,81.028,66.20%,81.10%,53.50%


In [50]:
# Scores by School Size
bins_size = [0,1000 ,2000, 5000] # create bins to cut the data
group_size = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"] # what values will be contained in the bins

# subset of school summary dataframe
score_size = school_summary_df[["Total Students",
                                "Average Math Score",
                               "Average Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing"                              
                                                             ]]

# Creating a new column and adding the bins in based on the cut values
score_size["School Size"] = pd.cut(score_size["Total Students"],bins_size,labels=group_size,include_lowest=True)

# grouping the new data set based on the newly created column
score_size_df = round(score_size.groupby("School Size").mean(),3)
score_size_df[["Average Math Score",
                               "Average Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing"]]

# formatting
score_size_df["% Passing Math"] = score_size_df["% Passing Math"].map("{:,.2%}".format)
score_size_df["% Passing Reading"] = score_size_df["% Passing Reading"].map("{:,.2%}".format)
score_size_df["% Overall Passing"] = score_size_df["% Overall Passing"].map("{:,.2%}".format)


score_size_df

<ipython-input-50-db19140db777>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_size["School Size"] = pd.cut(score_size["Total Students"],bins_size,labels=group_size,include_lowest=True)


,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,,
Small (<1000),694.500,83.822,83.930,93.60%,96.10%,89.90%
Medium (1000-2000),1704.400,83.375,83.864,93.60%,96.80%,90.60%
Large (2000-5000),3657.375,77.746,81.344,70.00%,82.80%,58.30%


In [51]:
# Scores by School Type
# no need to create bins - there are only 2 types of Schools
# subset of school summary dataframe
score_type = school_summary_df[["School Type",
                                "Average Math Score",
                               "Average Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing"                              
                                                             ]]

# grouping the new data set based on the newly created column
score_type_df = round(score_type.groupby("School Type").mean(),3)
score_type_df[["Average Math Score",
                               "Average Reading Score",
                               "% Passing Math",
                               "% Passing Reading",
                               "% Overall Passing"]]

# formatting
score_type_df["% Passing Math"] = score_type_df["% Passing Math"].map("{:,.2%}".format)
score_type_df["% Passing Reading"] = score_type_df["% Passing Reading"].map("{:,.2%}".format)
score_type_df["% Overall Passing"] = score_type_df["% Overall Passing"].map("{:,.2%}".format)


score_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.474,83.896,93.60%,96.60%,90.40%
District,76.957,80.967,66.50%,80.80%,53.70%
